# Prediction on OpenBCI data
___

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
import joblib
import numpy as np

from sklearn.metrics import (accuracy_score,
                             confusion_matrix,
                             plot_confusion_matrix,
                             classification_report,
                             f1_score,
                             cohen_kappa_score,
                             log_loss)

from scipy.signal import medfilt

from hmmlearn.hmm import MultinomialHMM
from constants import (SLEEP_STAGES_VALUES, N_STAGES, EPOCH_DURATION)
from models.model_utils import print_hypnogram

In [ ]:
X = np.load("data/X_openbci.npy", allow_pickle=True)
y = np.load("data/y_openbci.npy", allow_pickle=True)
X = np.vstack(X)
y = np.hstack(y)

print(X.shape)
print(y.shape)

## Load classifier
___

In [ ]:
MODELS_DIR = "trained_model"

classifier = joblib.load(f"{MODELS_DIR}/classifier_RF.joblib")

## Prediction
___

In [ ]:
y_pred = classifier.predict(X)

In [ ]:
print(confusion_matrix(y, y_pred))

print(classification_report(y, y_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y, y_pred))

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)
print_hypnogram([y_pred, y], labels=["Classifier", "Alexandra"], subject=0, night=0)

## Median filter
___

In [ ]:
KERNEL_SIZE=3

y_medfilt_pred = medfilt(y_pred, kernel_size=KERNEL_SIZE)

In [ ]:
print(confusion_matrix(y, y_medfilt_pred))

print(classification_report(y, y_medfilt_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y, y_medfilt_pred))

print_hypnogram([y_medfilt_pred, y], labels=[ "Classifier (Medfilter)", "Alexandra"], subject=0, night=0)

## Hidden Markov Model
___

In [ ]:
hmm_model = MultinomialHMM(n_components=N_STAGES)


hmm_model.emissionprob_ = np.load(f"{MODELS_DIR}/HMM_emissionprob.npy", allow_pickle=True)
hmm_model.startprob_ = np.load(f"{MODELS_DIR}/HMM_startprob.npy", allow_pickle=True)
hmm_model.transmat_ = np.load(f"{MODELS_DIR}/HMM_transmat.npy", allow_pickle=True)

y_hmm_pred = hmm_model.predict(y_pred.reshape(-1, 1))

In [ ]:
print(confusion_matrix(y, y_hmm_pred))

print(classification_report(y, y_hmm_pred, target_names=SLEEP_STAGES_VALUES.keys()))

print("Agreement score (Cohen Kappa): ", cohen_kappa_score(y, y_hmm_pred))

print_hypnogram([y_hmm_pred, y], labels=[ "Classifier (HMM)", "Alexandra"], subject=0, night=0)